# 문제 3-1

In [12]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])


sk


In [13]:

# %pip install langchain_community faiss-cpu
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from pprint import pprint

In [14]:
pythontext_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,        #  법률 사례는 1200-1800자 권장
    chunk_overlap=300,      #  사례 맥락 보존을 위해 200-400자
    separators=[
        "\n【사건개요】",     #  법률 문서 섹션 구분자
        "\n【쟁점사항】",     #  쟁점 부분 구분
        "\n【처리경위】",     #  처리 과정 구분
        "\n【처리결과】",     #  결과 부분 구분
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)


In [15]:
pythonembeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",  #  한국어 법률 용어에 최적
    dimensions=1536                  #  고성능 임베딩 차원
)


In [16]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('../data/콘텐츠분쟁해결_사례.pdf')
documents = loader.load()

chunks = pythontext_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(
    model = "text-embedding-3-large",
    dimensions=1536
)


vectorstore = FAISS.from_documents(chunks, embeddings)


pythonretriever = vectorstore.as_retriever(
    search_type="similarity",        #  또는 "mmr" (다양성 고려시)
    search_kwargs={"k": 5}        #  상위 5개 관련 사례 검색
)


In [17]:
pythonllm = ChatOpenAI(
    model="gpt-4o",                 #  또는 "gpt-4o-mini" (경제적)
    temperature=0.2,                #  법률 조언은 정확성 중시
    max_tokens=2000                 #  충분한 답변 길이
)


In [18]:
pythonprompt_template = """
당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요                    # 사례 기반 답변
2. 관련 법령이나 조항이 있다면 명시하세요               # 법적 근거 제시
4. 실무적 해결방안을 단계별로 제시하세요               #  단계쩍 해결방안 마련
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요    # 유사 사례 참조
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요  #  한계 인정

전문 법률 조언:"""



In [19]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


prompt = PromptTemplate(
    template = pythonprompt_template,
    input_variables=["content", "question"]
)

pythonqa_chain = RetrievalQA.from_chain_type(
    llm=pythonllm,                        #  위에서 설정한 LLM
    chain_type="stuff",             #  문서들을 하나로 합쳐서 처리
    retriever=pythonretriever,            #  위에서 설정한 검색기
    chain_type_kwargs={"prompt": prompt},  #  법률 자문 프롬프트
    return_source_documents=True    #  참조 문서도 함께 반환
)


In [20]:
pythontest_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

In [21]:
for i, question in enumerate(pythontest_questions, 1):
    print(f"\n【테스트 {i}/5】")
    print(f" 질문: {question}")
    print(" 답변 생성 중...")
    
    # RAG 실행
    result = pythonqa_chain.invoke({"query": question})
    answer = result["result"]
    source_docs = result["source_documents"]
    
    print(f"\n 답변:")
    print("-" * 50)
    print(answer)
    
    # 참조 문서 정보
    print(f"\n 참조 문서:")
    for j, doc in enumerate(source_docs[:3], 1):
        page = doc.metadata.get('page', 'N/A')
        preview = doc.page_content[:80].replace('\n', ' ')
        print(f"   {j}. 페이지 {page}: {preview}...")
    
    print("\n" + "-" * 40)



【테스트 1/5】
 질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
 답변 생성 중...

 답변:
--------------------------------------------------
온라인 게임에서 시스템 오류로 인해 아이템이 사라진 경우, 게임회사가 복구를 거부하는 상황에 대한 법률 조언을 다음과 같이 드리겠습니다.

### 사례 기반 답변
제시된 사례들 중 시스템 오류로 인한 아이템 소실과 관련된 사례는 다음과 같습니다:

1. **사례 1 (2006년)**: 시스템 오류로 아이템이 소멸되었으나, 계정 명의자가 아니어서 복구가 불가하다는 사례.
2. **사례 3 (2009년)**: 시스템 오류로 게임머니가 소실되었으나, 시스템 오류가 발생하지 않았다는 게임사의 주장으로 복구가 거부된 사례.

### 법적 근거 제시
- **계약법**: 게임 이용 약관은 계약의 일종으로, 약관에 명시된 내용에 따라 책임이 결정될 수 있습니다. 약관에 시스템 오류로 인한 복구 조항이 있는지 확인이 필요합니다.
- **소비자 보호법**: 소비자는 정상적인 서비스 이용 중 발생한 피해에 대해 보상을 받을 권리가 있습니다. 시스템 오류가 게임사의 책임이라면 소비자는 보상을 요구할 수 있습니다.

### 유사 사례 참조
- **사례 1**에서는 계정 명의자가 아니어서 복구가 불가하다는 결론이 나왔습니다. 이는 계정 명의와 실소유주가 다를 경우 복구가 어렵다는 점을 보여줍니다.
- **사례 3**에서는 시스템 오류가 발생하지 않았다는 게임사의 주장으로 복구가 거부되었습니다. 이는 시스템 오류가 명확히 입증되지 않으면 복구가 어려울 수 있음을 시사합니다.

### 단계적 해결방안 마련
1. **약관 검토**: 게임사의 이용 약관을 검토하여 시스템 오류로 인한 복구 조항이 있는지 확인합니다.
2. **증거 수집**: 시스템 오류가 발생했음을 입증할 수 있는 증거(스크린샷, 로그 기록 등)를 수집합니다.
3